In [1]:
from os import listdir, remove
from os.path import join
from json import load
from tifffile import imread,imwrite
from re import search, findall

import numpy as np
import pandas as pd

# Notebook Explanation
## Documentation for Data Preprocessing
This notebook consists of two separate processes: (1) creating the `.csv` files for the chemical nodes, action nodes and links and (2) creating the `.cypher` file to load the `.csv` files into Neo4j database. After creating the `.cypher` file, the script deletes all `.csv` files that were used for imports.
1. The detailed documentation for step (1) is in the notebook `preprocess explanation 1.ipynb`.
2. The detailed documentation for step (2) is in the notebook `preprocess explanation 2.ipynb`.

## Specify the DBMS storage folders
1. `filepath_import` specifies the import folder inside the DMBS folder. We will need to save the `.csv` files inside this folder for the `.cypher` to load all the data into the nodes.
2. `filepath_bin` specifies the bin folder inside the DBMS folder. When we start a Database on Neo4j Desktop, we need to put the `.cypher` file inside DMBS bin folder for the terminal to read the `cypher shell`.
3. `filepath_all_batches` specifies the location of all `psk` and `char` folders with the JSON worklist and Characterization folder.

In [2]:
filepath_import = "relate-data\\dbmss\\dbms-ea5729e6-f244-4c0d-b059-176025203af9\\import"
filepath_bin = "relate-data\\dbmss\\dbms-ea5729e6-f244-4c0d-b059-176025203af9\\bin"
filepath_all_batches = "files"
version_control_file = 'version_control.csv'

# Step 1: Creating `.csv` files
## Creating `action.csv` files
### Helper functions for `action.csv`

In [3]:
# PIPELINE
def step_helper(worklists):
    """
    Helper function to get the step names for each step in worklist(s).
    Takes either a single worklist or a list of worklists in as input and returns
    a list of lists, with each list of steps corresponding to an input worklist.
    """
    steps = []
        
    # iterate through worklists and get steps e.g. destination, drops, etc
    for w in worklists:
        w_steps = []
        for step in w:
            w_steps.append(list(step['details'].keys())[0])
        steps.append(w_steps)
    
    return steps

def dissolve_helper(drop_list, step_num = None):
    """
    Helper function that takes in a list of drop steps and .
    """
    drop_steps = drop_list['details']['drops']
    
    if drop_steps[0]['solution']['solutes'] != '':
        num_chems = len(drop_steps[0]['solution']['solutes'].split('_'))
        num_chems += len(drop_steps[0]['solution']['solvent'].split('_'))
    else:
        num_chems = 1
    actions = []
    for i in range(1, num_chems+1):
        row = {}
        row['step_id'] = i
        row['action'] = 'dissolve'
        row['chemical_from'] = i
        actions.append(row)
    return actions    

def drop_helper(drop_list, step_num):
    """
    Helper function to process drop steps in a worklist
    """
    drop_list = drop_list['details']['drops']
    
    # exclude volume and solution information, as this is handled when 
    # creating the chem csvs/nodes
    exclude_attributes = ['volume', 'solution']
    row_attributes = [i for i in drop_list[0].keys() if i not in exclude_attributes]
    drop_rows = []
    i = 1
    for drop in drop_list:
        row = {}
        for j in row_attributes:
            row['step_id'] = step_num * 2 + i
            row['action'] = 'drop'
            row['chemical_from'] = step_num + i
            row["drop_"+j] = drop.get(j)
        i += 1
        drop_rows.append(row)
    return drop_rows

def spin_helper(spin_list, step_num):
    """
    Helper function to process spin steps in a worklist
    """
    spin_rows = []
    spin_details = spin_list['details']['steps']
    for spin in spin_details:
        row = {}
        row['step_id'] = step_num
        row['action'] = 'spin'
        for i in spin:
            row["spin_"+i] = spin.get(i)
        step_num += 2
        
        attributes = ['start', 'start_actual', 'finish_actual', 'liquidhandler_timings', 'spincoater_log']
        for i in attributes:
            if i in spin_list.keys():
                if (i == 'spincoater_log') and ('rpm' in spin_list[i]):
                    log_attr = spin_list[i].keys()
                    for j in log_attr:
                        row['spin_log_'+j] = spin_list[i][j]
                else:
                    row[i] = spin_list[i]
        spin_rows.append(row)
    return spin_rows

def anneal_helper(anneal_list, step_num):
    """
    Helper function to process anneal steps in a worklist
    """
    row = []
    anneal_info = anneal_list['details']
    anneal_row = {'step_id':step_num, 'action': 'anneal'}
    for i in anneal_info:
        anneal_row['anneal_'+i] = anneal_info.get(i)
        
    attributes = [i for i in anneal_list.keys() if i not in ['precedent', 'id', 'details']]
    for i in attributes:
        anneal_row[i] = anneal_list[i]
    row.append(anneal_row)
    return row

def rest_helper(rest_step, step_num):
    """
    Helper function to process rest steps in a worklist
    """
    rest_row = {'step_id': step_num, 'action':'rest'}
    rest_row['rest_duration'] = rest_step['details']['duration']
    
    attributes = [i for i in rest_step.keys() if i not in ['precedent', 'id', 'details']]
    for i in attributes:
        rest_row[i] = rest_step[i]
    return [rest_row]

def char_helper(char_list, step_num):
    """
    Helper function to process characterization steps in a worklist
    """
    char_rows = []
    char_details = char_list['details']['characterization_tasks']
    for char in char_details:
        char_params = [i for i in list(char['details'].keys())]
        char_info = [i for i in char if i != 'details']
        row = {
            'step_id':step_num, 
            'action':'char'
        }
        for i in char_params:
            row['char_'+i] = char['details'][i]
        for i in char_info:
            if type(char[i]) == str:
                if i == 'name':
                    row['char_'+i] = char[i].lower().split("_")[0]
                else:
                    row['char_'+i] = char[i].lower()
            else:
                row['char_'+i] = char[i]
                
        attributes = [i for i in char_list.keys() if i not in ['precedent', 'id', 'details', 'name', 'sample', ]]
        for i in attributes:
            row[i] = char_list[i]
        char_rows.append(row)
        step_num += 2
        
    return char_rows

In [4]:
# PIPELINE: function map used in action_table to call the helper functions above
func_map = {
    "dissolve": dissolve_helper,
    "drops": drop_helper,
    "spin": spin_helper,
    "anneal": anneal_helper,
    "duration": rest_helper,
    "characterization_tasks": char_helper
}

### Processing `action.csv` file

In [5]:
# PIPELINE
def action_table(worklists, sample_id=np.nan, batch_id=np.nan):
    """
    The action_table function takes in one or more worklists. 
    If there are multiple worklists, the first worklist will be the
    psk worklist and the second will be the char worklist.
    """
    if type(worklists[0]) != list:
        worklists = [worklists]
        
    # obtain steps from worklists
    steps = step_helper(worklists)
    
    rows = []
    step_num = 1
    for i in range(len(worklists)):
        curr_worklist = worklists[i]
        curr_steplist = steps[i]
        for j in range(len(curr_steplist)):
            if curr_steplist[j] == 'destination':
                continue
            else:
                if curr_steplist[j] == 'drops':
                    rows = rows + func_map['dissolve'](curr_worklist[j], step_num)
                    step_num = rows[-1]['step_id']
                    rows = rows + func_map['drops'](curr_worklist[j], step_num)
                    step_num = rows[-1]['step_id']+3
                    rows = rows + func_map['spin'](curr_worklist[j], step_num)
                    step_num = rows[-1]['step_id']+2
                else:
                    if curr_steplist[j] == 'duration' and curr_worklist[j]['name'] == 'anneal':
                        rows = rows + func_map['anneal'](curr_worklist[j], step_num)
                    else:
                        rows = rows + func_map[curr_steplist[j]](curr_worklist[j], step_num)
                    step_num = rows[-1]['step_id']+2
    
    res = pd.DataFrame(rows)
    res['sample_id'] = [sample_id] * res.shape[0]
    res['batch_id'] = [batch_id] * res.shape[0]
    
    return res

## Characterization Task
Need this as an input for the `link.csv` file.
### Formatting `.tif` images to a node

In [6]:
# helper function used in char_outputs.
def load_image(fid):
    img = imread(fid) * 64 * 255
    img = img.astype(np.float32)
    img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140]) #convert to single channel/greyscale??
    return img.astype(int) #truncate

In [7]:
# PIPELINE
def char_outputs(folder, sample):
    path = folder + '/' + sample + "/characterization0"
    fids = [f for f in listdir(path)]
    
    data = []
    ids = []
    for fid in fids:
        if '.tif' in fid:
            data.append(load_image(path+"/"+fid))
        elif '.csv' in fid:
            data.append(pd.read_csv(path+"/"+fid).drop(0,axis=0).to_dict())
        else:
            print('haven\'t had to deal w this filetype yet')

        ids.append(fid.split('_', 1)[1].split('.')[0])    
    df = pd.DataFrame({'join_on': ids, 'fid':fids, 'output':data})

    return(df)

In [8]:
def append_outputs(output_df, action_df):
    """
    Helper function to append the characterization outputs as nodes to the action table
    """
    step_id = action_df.iloc[-1]['step_id']+2
    
    row_template = action_df.iloc[-1].copy()
    row_template.loc[:]=np.nan
    row_template['action'] = 'char_output'
    row_template['sample_id'] = action_df['sample_id'].iloc[0]
    row_template['batch_id'] = action_df['batch_id'].iloc[0]
    
    output_rows = []
    for r in range(output_df.shape[0]):
        output_row = output_df.iloc[r]
        row = row_template.copy()
        row['step_id'] = step_id
        row['char_name'] = output_row['join_on']
        row['fid'] = output_row['fid']
        row['output'] = output_row['output']
        output_rows.append(row)
        step_id += 1
        
    action_df = action_df.append(output_rows)
    
    return action_df   

In [9]:
def append_metrics(action_df, metric_row):
    """
    Helper function that appends the fitted characterization metrics 
    as a node to the end of each sample's graph.
    """
    step_id = action_df.iloc[-1]['step_id']+2
    
    row_template = action_df.iloc[-1].copy()
    row_template.loc[:]=np.nan
    row_template['step_id'] = step_id
    row_template['action'] = 'fitted_metrics'
    row_template['sample_id'] = action_df['sample_id'].iloc[0]
    row_template['batch_id'] = action_df['batch_id'].iloc[0]
    
    for col in metric_row:
        if 'name' in col:
            pass
        else:
            row_template[col] = metric_row[col].iloc[0]
    
    action_df = action_df.append(row_template)
    
    return action_df

### Saving the `action.csv` file

In [10]:
# takes in samples, a list of json objects. 
# samples[0] is the psk worklist, samples[1] is the characterization worklist, if it exists.
def save_action_csv(samples, batch_id, filepath = '', char_data=None, metrics=None):
    """
    Function to parse worklist(s) in order to create action nodes.
    char_data and metrics are optional parameters that are passed into the
    function when such data exists and will be added to the sample's graph of data.
    """
    action_dfs = []
    for sample in samples[0]:
        if samples[1] != None:
            a_df = pd.DataFrame(action_table([samples[0][sample]['worklist'],
                                             samples[1][sample]['worklist']],
                                            sample, batch_id))
        else:
            a_df = pd.DataFrame(action_table([samples[0][sample]['worklist']], sample, batch_id))
            
        if char_data != None:
            folder = 'files/' + char_data + '/Characterization'
            output_df = char_outputs(folder, sample)
            a_df = append_outputs(output_df, a_df)
            
        if isinstance(metrics, pd.DataFrame):
            a_df = append_metrics(a_df, metrics[metrics['name'] == sample])
            
        a_df = a_df.astype({'chemical_from':'Int64'})
        action_dfs.append(a_df)
        fname = batch_id + '_' + sample + '_action.csv'
        fname = fname.replace(' ', '_')
        a_df.to_csv(join(filepath,fname),index=False)
    return action_dfs

## Creating `link.csv` file
### Helper functions for `link.csv`
Links and link helper functions used to create the table with link information; used to link chemicals and nodes together. 

In [11]:
# PIPELINE
def goes_into_links(dissolve_rows):
    """
    Helper method to create links for chemical nodes that "go into" a dissolve node.
        AKA the "GOES_INTO" links
    :param dissolve_rows: From an action table, takes in the rows of the table where dissolve
        steps are involved. For this batch, this is at the beginning steps of the action table
    """
    # step_id, action(link), chemical_from, step_to, chemical_to, step_from, sample_id, batch_id
    row_template = [0, 'GOES_INTO', 0, 0, np.nan, np.nan]
    links = []
    for i in range(dissolve_rows.shape[0]):
        row = dissolve_rows.iloc[i]
        link = row_template.copy()
        link[0] = row['step_id']
        link[2] = row['chemical_from']
        link[3] = link[2]
        links.append(link)
    return links

def output_links(dissolve_rows):
    """
    Helper method to create the OUTPUTS links from the initial solutes and solvents used
    :param dissolve_rows: From an action table, takes in the rows of the table where dissolve
        steps are involved. For this batch, this is at the beginning steps of the action table
    """
    row_template = [0, 'OUTPUTS', np.nan, np.nan, 0, 0]
    links = []
    mix_step = dissolve_rows.iloc[-1]['step_id']+1
    prev_step = dissolve_rows.iloc[-1]['step_id']
    for i in range(dissolve_rows.shape[0]):
        row = dissolve_rows.iloc[i]
        link = row_template.copy()
        link[0] = prev_step+1
        prev_step +=1
        link[4] = mix_step
        link[5] = row['step_id']
        links.append(link)
    return links

def next_func(other_steps, step_num):
    """
    Helper method to create NEXT links. These links are pretty generic and indicate either:
    1. Movement from one step to another
    or
    2. Movement from one task to the fitted metrics."""
    # step_id, action, chemical_from, step_to, chemical_to, step_from
    row_template = [0, 'NEXT', np.nan, np.nan, np.nan, 0]
    rows = []
    step_id = step_num
    if isinstance(other_steps, pd.Series):
        step_to = other_steps['step_id']
        row = row_template.copy()
        row[0] = step_id
        row[3] = step_to
        row[-1] = row[3] - 2
        rows.append(row)
    else:
        for i in range(other_steps.shape[0]):
            step_to = other_steps.iloc[i]['step_id']
            row = row_template.copy()
            row[0] = step_id
            row[3] = step_to
            row[-1] = row[3] - 2
            rows.append(row)
            step_id += 2
    return rows

def char_link_helper(output_rows, char_nodes, step_id):
    """
    Helper method to create NEXT links for characterization output nodes.
    """
    # step_id, action, chemical_from, step_to, chemical_to, step_from
    row_template = [0, 'NEXT', np.nan, np.nan, np.nan, 0]
    rows=[]

    for i in range(output_rows.shape[0]):
        curr_output = output_rows.iloc[i]
        char = curr_output['char_name']
        if '_' in char:
            char = 'plimaging'
        curr_node = char_nodes[char_nodes['char_name']==char]
        step_to = curr_output['step_id']
        row = row_template.copy()
        row[0] = step_id
        row[3] = step_to
        row[-1] = curr_node['step_id'].iloc[0]
        rows.append(row)
        step_id += 1
    return rows

### Processing `link.csv` file

In [12]:
def link_table(action_table, sample_id, batch_id):
    """
    Function that creates the table with link/relationship information between nodes.
    """
    link_cols = ['step_id',
    'action',
    'chemical_from',
    'step_to',
    'chemical_to',
    'step_from',
    'sample_id',
    'batch_id']
    
    # retrieve rows from the action_table where the action is dissolve
    dissolve_rows = action_table[action_table['action'] == 'dissolve']
    # create GOES_INTO links for the dissolve nodes, which link chemicals to dissolve nodes.
    go_into = goes_into_links(dissolve_rows)
    # create OUTPUTS links for the dissolve nodes, linking them to a chemical node (a new mix)
    outputs = output_links(dissolve_rows)
    
    # variable used to track step_id for links.
    next_link_step = outputs[-1][0] + 3
    
    # links solution from mix1 to the drop step
    mix1_into_drop = [next_link_step, 'GOES_INTO', outputs[-1][4], outputs[-1][0]+1, np.nan, np.nan]
    
    # increment link step
    next_link_step += 1 
    
    
    first_drop_step_id = action_table[action_table['action']=='drop'].iloc[0]['step_id']
    second_drop_step_id = first_drop_step_id + 1
    mix2_chem_id = mix1_into_drop[2] + 2
    
    mix1_to_mix2 = [next_link_step, 'NEXT', np.nan, np.nan, mix2_chem_id, first_drop_step_id]
    
    mix_antisolvent = [action_table.iloc[-1]['step_id']+4, 'GOES_INTO', mix2_chem_id-1, second_drop_step_id, np.nan, np.nan]
    
    next_link_step += 2
    drop_to_mix2 = [next_link_step, 'NEXT', np.nan, np.nan, mix2_chem_id, second_drop_step_id]
    
    spin_step_id = action_table[action_table['action']=='spin'].iloc[0]['step_id']
    ######
    next_link_step += 2
    mix2_to_spin = [next_link_step, 'GOES_INTO', mix2_chem_id, spin_step_id, np.nan, np.nan]
    
    if 'fid' in action_table.columns:
        next_steps = action_table.loc[(action_table['step_id']>spin_step_id) & (pd.isna(action_table['fid']))]
    else:
        next_steps = action_table.loc[(action_table['step_id']>spin_step_id)]
        
    next_rows = next_func(next_steps, next_link_step+2)
    
    # for characterization outputs
    if 'fid' in action_table.columns:
        output_rows = action_table.loc[~pd.isna(action_table['fid'])]
        char_rows = action_table.loc[(action_table['action'] == 'char') & (pd.isna(action_table['fid']))]
        char_output_links = char_link_helper(output_rows, char_rows, next_rows[-1][0]+2)
        
        res = go_into + outputs + [mix1_into_drop] + [mix1_to_mix2] + [mix_antisolvent] + [drop_to_mix2] + [mix2_to_spin] + next_rows + char_output_links 

    else:
        res = go_into + outputs + [mix1_into_drop] + [mix1_to_mix2] + [mix_antisolvent] + [drop_to_mix2] + [mix2_to_spin] + next_rows
        
    for i in res:
        i.append(sample_id)
        i.append(batch_id)
        
    return pd.DataFrame(res, columns=link_cols)

### Saving `link.csv` file

In [13]:
def save_link_csv(samples, batch_id, filepath = ''):
    link_dfs = []
    for act in samples:
        l_df = link_table(act, act.iloc[0]['sample_id'], batch_id)
        l_df = l_df.astype({'chemical_from':'Int64', 'step_to':'Int64', 'chemical_to':'Int64', 'step_from':'Int64'})
        link_dfs.append(l_df)
        fname = batch_id + '_' + act.iloc[0]['sample_id'] + '_link.csv'
        fname = fname.replace(' ', '_')
        l_df.to_csv(join(filepath, fname), index=False)
    return link_dfs

## Creating `chem.csv` file
### Helper functions for `chem.csv`

In [14]:
def split_chemicals(input_string):
    result = []
    for i in input_string.split('_'):
        first_digit = search(r'\d+.\d+', i).start()
        result.append((i[:first_digit], float(i[first_digit:])))
    return result

def create_new_row(**kwargs):
    return dict(zip(kwargs, kwargs.values()))

def check_name_format(chemicals_string_list):
    """Function to check if the chemicals from drop step follows 
    the format 'First0.75_Second0.10_Third0.5_Fourth0.5' 
    return True if yes and False if no
    """
    return search(r'[A-Za-z]+\d+.?\d+_', chemicals_string_list) != None

### Processing `chem.csv` file

In [15]:
def chem_table(sample, batch_id):
    chem_cols = ['chemical_id', 'batch_id', 'content', 'concentration', 'molarity', 'volume', 'chem_type']
    chem = pd.DataFrame(columns = chem_cols)
    
    # the first chemical has the id of 1, the first mix has the id of 1
    chemical_id = 1
    mix_id = 1
    
    sample_id = sample['name']
    worklist = sample['worklist']
    for step in worklist:
        # check if the 'details' is a key in worklist and check if steps is in the drops
        # if yes, go check the content for the chemical
        # if not, then move on since there is no chemical for mixing
        if 'details' in step and 'drops' in step['details']:
            for droplet in step['details']['drops']:
                # if it has both solvent and solute
                if 'solution' in droplet and droplet['solution']['solutes'] != '' and droplet['solution']['solvent'] != '':
                    # check if the solutes string follows the format to further breaking 
                    # it down using check_name_format helper function
                    # if yes, break the string down using the split_chemicals to get the name and the concentration
                    if check_name_format(droplet['solution']['solutes']):
                        for solute in split_chemicals(droplet['solution']['solutes']):
                            content, concentration = solute
                            if ((chem['content'] == content) & (chem['concentration'] == concentration)).sum() == 0:
                                new_row = create_new_row(chemical_id = chemical_id, batch_id = batch_id, 
                                                         content = content, concentration = concentration, 
                                                         chem_type = 'solute', sample_id = sample_id)

                                chem = chem.append(new_row, ignore_index=True)
                                chemical_id += 1
                    # if no, use the solute recipe name as the content (such as 'Xu-Recipe-PSK')
                    else:
                            new_row = create_new_row(chemical_id = chemical_id, batch_id = batch_id, 
                                                     content = droplet['solution']['solutes'], 
                                                     chem_type = 'solute', sample_id = sample_id)

                            chem = chem.append(new_row, ignore_index=True)
                            chemical_id += 1
                    # check if the antisolvent string follows the format to further breaking 
                    # it down using check_name_format helper function
                    # if yes, break the string down using the split_chemicals to get the name and the concentration
                    if check_name_format(droplet['solution']['solvent']):
                        for solvent in split_chemicals(droplet['solution']['solvent']):
                            content, concentration = solvent
                            if ((chem['content'] == content) & (chem['concentration'] == concentration)).sum() == 0:
                                new_row = create_new_row(chemical_id = chemical_id, batch_id = batch_id, 
                                                         content = content, concentration = concentration, 
                                                         chem_type = 'solvent', sample_id = sample_id)
                                chem = chem.append(new_row, ignore_index=True)
                                chemical_id += 1
                    # if no, use the solute recipe name as the content (such as 'Xu-Recipe-PSK')
                    else:
                        new_row = create_new_row(chemical_id = chemical_id, batch_id = batch_id, 
                                                 content = droplet['solution']['solutes'], 
                                                 chem_type = 'solvent', sample_id = sample_id)

                        chem = chem.append(new_row, ignore_index=True)
                        chemical_id += 1
                    # adding the mix (or solution) from the previous solvents and solutes
                    new_row = create_new_row(chemical_id = chemical_id, batch_id = batch_id, 
                                             content = 'Mix'+str(mix_id), volume = droplet['volume'], 
                                             molarity = droplet['solution']['molarity'],
                                             chem_type = 'solution', sample_id = sample_id)

                    chem = chem.append(new_row, ignore_index=True)
                    mix_id += 1
                    chemical_id += 1
                # check if the drop is an antisolvent (no solvent and no solute present)
                if 'solution' in droplet and droplet['solution']['solutes'] == '':
                    # check if the antisolvent is already in the df, add to the df if not in the df
                    if (chem['content'] == droplet['solution']['solvent']).sum() == 0:
                        new_row = create_new_row(chemical_id = chemical_id, batch_id = batch_id, 
                                                 content = droplet['solution']['solvent'],
                                                 molarity = droplet['solution']['molarity'],
                                                 chem_type = 'antisolvent', sample_id = sample_id)
                        chem = chem.append(new_row, ignore_index=True)
                        chemical_id += 1
                        
                    # adding the mix
                    new_row = create_new_row(chemical_id = chemical_id, batch_id = batch_id, 
                                             content = 'Mix'+str(mix_id),
                                             volume = droplet['volume'],
                                             chem_type = 'solution', sample_id = sample_id)

                    chem = chem.append(new_row, ignore_index=True)
                    mix_id += 1
                    chemical_id += 1
    return chem

### Saving `chem.csv` file

In [16]:
def save_chem_csv(samples, batch_id, filepath = ''):
    """Takes in the dictionary of samples and run chem_table and save the resulting csv files.
    Replaces the whitespace in the name with underscore for Neo4J compatibility"""
    for sample in samples:
        filename = batch_id + '_' + sample + '_' + 'chem.csv'
        filename = filename.replace(' ', '_')
        chem_table(samples[sample], batch_id).to_csv(join(filepath,filename), index=False)

## Saving `.csv` files in Neo4j import folder
### Helper functions to find the files and folders

In [17]:
def find_psk_files(directory = 'files'):
    """Takes in the name of the folder with all of the psk batches.
    Returns a list of tuples with the batch name and the maestro JSON file.

    Parameters
    ----------
    directory: str
        The name of the folder with all of the psk batches.

    Returns
    -------
    list
        A list of tuples with first element being the batch name and the second element 
        being the full filepath of the maestro JSON file.
        
    Example
    -------
    >>> find_psk_json_files('files')
    [('20230110_B18-psk', 'files\\20230110_B18-psk\\maestro_sample_log.json'),
     ('20230114_B18-psk_1', 'files\\20230114_B18-psk_1\\maestro_sample_log.json')]
    """
    json_files = []
    # going to every folder to find all the psk folders with -psk suffix
    for folder in listdir(directory):
        if search(r'-psk(_\d)*$', folder) != None:
            # then check if the .json file is available in each of the psk folder
            for file in listdir(join(directory, folder)):
                if file.endswith('.json'):
                    json_files.append((folder, join(directory, folder, file)))
    return json_files

def find_char_file(psk_batch, directory = 'files'):
    try:
        batch_char_folders = []
        batch = search(r'(?:\d+_)(\w+)', psk_batch).group(1)
        for char_folder_name in listdir(directory):
            if batch + '-char' in char_folder_name:
                if search(r'\d+$', char_folder_name) != None:
                    char_number = int(search(r'\d+$', char_folder_name).group(0))
                    batch_char_folders.append((char_number, char_folder_name))
                else:
                    batch_char_folders.append((0, char_folder_name))
        return sorted(batch_char_folders, key=lambda x: x[0], reverse=True)[0][1]
    except:
        return None
    
def find_fitted_char_metrics(psk_batch, directory = 'files'):
    for file in listdir(join(directory, find_char_file(psk_batch, directory))):
        if ('fitted_characterization' in file or 'characterization_metrics' in file) and '.csv' in file:
            return join(directory, find_char_file(psk_batch, directory), file)
    return None

def find_new_batches(filepath_all_batches, version_control_file = 'version_control.csv'):
    psk_folders_files = find_psk_files(filepath_all_batches)
    all_batches = [i[0] for i in psk_folders_files]
    current_batches = pd.read_csv(version_control_file)['current_batches'].to_list()
    
    new_batches = []
    for folder, file in psk_folders_files:
        if folder not in current_batches:
            new_batches.append((folder, file))
            
    pd.DataFrame(all_batches, columns = ['current_batches']).to_csv(version_control_file, index=False)
    return new_batches

In [18]:
find_psk_files()

[('20221206_B17-psk', 'files\\20221206_B17-psk\\maestro_sample_log.json'),
 ('20230114_B18-psk_1', 'files\\20230114_B18-psk_1\\maestro_sample_log.json'),
 ('20230122_B19-psk', 'files\\20230122_B19-psk\\maestro_sample_log.json')]

## Saving `.csv` files

In [19]:
new_batches = find_new_batches(filepath_all_batches, version_control_file)

action_df_list = []
for batch_id, filename in new_batches:
    # handle char folder/files, if they exist
    char_file = find_char_file(batch_id, filepath_all_batches)
    char_data = None
    if char_file != None:
        f = open(join(filepath_all_batches, char_file, 'maestro_sample_log.json'))
        char_data = load(f)
        f = f.close()
    
    metrics_file = find_fitted_char_metrics(batch_id, filepath_all_batches)
    metrics = None
    if metrics_file != None:
        metrics = pd.read_csv(metrics_file)
    
    f = open(filename)
    data = load(f)
    f = f.close()
    action_df_list = save_action_csv([data, char_data], batch_id, filepath_import, char_file, metrics)
    save_link_csv(action_df_list, batch_id, filepath_import)
    save_chem_csv(data, batch_id, filepath_import)

# Step 2: Graph Generator
After having all the necessary `.csv` files in the current directory, we use the script below to make graph on Neo4j.
## Helper functions

In [20]:
def find_columns(csv_file):
    """
    Helper method to find all the columns
    
    :param csv_file: Takes in the csv file string, e.g:
        'WBG Repeat, Batch 4 (Experiment 1)_sample12_action.csv'
        
    :return: Returns a list all the columns
    """
    df = pd.read_csv(csv_file)
    return df.columns.to_list()

def find_csv_files(filepath):
    """Find all the csv files ending with the name sample_chem.csv, sample_link.csv, 
    sample_action.csv within the current directory and group by sample"""
    csv_files = [x for x in listdir(filepath) if x.endswith('.csv')]
    unique_samples = list(set(findall('([\w-]+sample\d+)', ' '.join(csv_files))))

    file_dict = {}
    file_list = []
    
    types = ['action', 'chem', 'link']

    for sample in unique_samples:
        csv_tables = [sample + '_' + i + '.csv' for i in types]
        file_list += csv_tables
        file_dict[sample] = dict(zip(types, csv_tables))
        
    return file_dict, file_list

## Create Action and Chem nodes

In [21]:
def create_node(filepath, node_type, cols, stored_folder = ''):
    query = "LOAD CSV WITH HEADERS FROM \"file:///"
    query += stored_folder
    
    if stored_folder != '':
        query += '/'
        
    filepath = filepath.split('\\')[-1]
    query += "{}\" ".format(filepath)
    if node_type == 'chem':
        query += "AS row CREATE (c:Chemical {"
    elif node_type == 'action':
        query += "AS row CREATE (a:Action {"
    
    # add the columns into the query
    query_columns = (["{}: row['{}'], ".format(cols[i], cols[i]) if i < len(cols) - 1 
           else "{}: row['{}']".format(cols[i], cols[i]) for i in range(len(cols))])
    query += ''.join(query_columns)
    
    # close the brackets and return
    return query + "});"

## Create Links

In [22]:
def create_links(filepath, stored_folder = ''):
    query_start = "LOAD CSV WITH HEADERS FROM \"file:///"
    query_start += stored_folder
    
    if stored_folder != '':
        query_start += '/'
    
    filepath = filepath.split('\\')[-1]
    query_start += "{}\" ".format(filepath)
    
    query_1 = "AS row MATCH (c:Chemical {chemical_id: row['chemical_from'], sample_id: row['sample_id'], \
batch_id: row['batch_id']}), (a:Action {step_id:row['step_to'], sample_id: row['sample_id'], \
batch_id: row['batch_id']}) CREATE (c)-[:GOES_INTO]->(a);"
    
    query_2 = "AS row MATCH (a1:Action {action: 'dissolve', step_id: row['step_from'], sample_id: row['sample_id'], \
batch_id: row['batch_id']}),(c1:Chemical {chemical_id:row['chemical_to'], sample_id: row['sample_id'], \
batch_id: row['batch_id']}) CREATE (a1)-[:OUTPUTS]->(c1);"
    
    query_3 = "AS row MATCH (a3:Action {action:'drop',step_id: row['step_from'], sample_id: row['sample_id'], \
batch_id: row['batch_id']}),(c4:Chemical {chemical_id:row['chemical_to'], sample_id: row['sample_id'], \
batch_id: row['batch_id']}) CREATE (a3)-[:NEXT]->(c4);"
    
    query_4 = "AS row MATCH (a3:Action {step_id: row['step_from'], sample_id: row['sample_id'], \
batch_id: row['batch_id']}),(a4:Action {step_id:row['step_to'], sample_id: row['sample_id'], \
batch_id: row['batch_id']}) CREATE (a3)-[:NEXT]->(a4);"
    

    queries = []
    for i in [query_1, query_2, query_3, query_4]:
        query_str = query_start + i
        queries.append(query_str)
    
    return queries

## Combining Action, Chem, and Link

In [23]:
def query_maker(chem_filepath, action_filepath, link_fileid, stored_folder = ''):
    """
    Calls the other query functions in this one function, given the necessary file ids"""
    queries = []
    
    chem_cols = find_columns(chem_filepath)
    queries.append(create_node(chem_filepath, 'chem', chem_cols, stored_folder))
    
    action_cols = find_columns(action_filepath)
    queries.append(create_node(action_filepath, 'action', action_cols, stored_folder))
    
    queries = queries + create_links(link_fileid, stored_folder)
    return queries

## Taking applicable `.csv` files to make `.cypher` file

In [24]:
file_dict, file_list = find_csv_files(filepath_import)
queries = []

for sample in file_dict:
    # the input file_id is in the order of chem.csv, action.csv, link.csv
    queries.append(query_maker(join(filepath_import, file_dict[sample]['chem']), 
                                join(filepath_import, file_dict[sample]['action']), 
                                join(filepath_import, file_dict[sample]['link']), 
                                stored_folder = '')) 

In [25]:
# saving the file as .cypher file
output = open(join(filepath_bin,'output.cypher'), 'w')
for q in queries:
    for query in q:
        output.write(query)
output.close()

## Deleting the applicable `.csv` file after making the `.cypher` file

In [26]:
for file in file_list:
    remove(join(filepath_import, file))